In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.koreabaseball.com/")

In [6]:
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR, "#popupVideo > img").click()

In [7]:
driver.find_element(By.CSS_SELECTOR, "#lnb > li:nth-child(3) > a").click()

In [8]:
driver.find_element(By.CSS_SELECTOR,"#cphContents_cphContents_cphContents_ddlTeam").click()

In [9]:
import time
import re
pattern = re.compile("playerId=([0-9]+)")
playid = []
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"
for x in range(2,12):
    for_1 = select_team.format(x)
    driver.find_element(By.CSS_SELECTOR, for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(By.CSS_SELECTOR, f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print (e)
        time.sleep(2)


In [10]:
playid

['53103',
 '51143',
 '67119',
 '53104',
 '53168',
 '66112',
 '51109',
 '66145',
 '77564',
 '53144',
 '52105',
 '69105',
 '67169',
 '68130',
 '64596',
 '50157',
 '68109',
 '66162',
 '52140',
 '75867',
 '51154',
 '50115',
 '67122',
 '76290',
 '69102',
 '68119',
 '79365',
 '53139',
 '51112',
 '62415',
 '63145',
 '65109',
 '78168',
 '68159',
 '50109',
 '67143',
 '50150',
 '53105',
 '51111',
 '73801',
 '68110',
 '65207',
 '65115',
 '52123',
 '52124',
 '67116',
 '53123',
 '79109',
 '53101',
 '50106',
 '63961',
 '50126',
 '51110',
 '65117',
 '69168',
 '61145',
 '68106',
 '69134',
 '53160',
 '69108',
 '67164',
 '61186',
 '53106',
 '53100',
 '61891',
 '61101',
 '67123',
 '69159',
 '79150',
 '51163',
 '52149',
 '76650',
 '79140',
 '65157',
 '62146',
 '65905',
 '52103',
 '65320',
 '68100',
 '69103',
 '52145',
 '50107',
 '63248',
 '50108',
 '77243',
 '53102',
 '52119',
 '66108',
 '53006',
 '69064',
 '68050',
 '50066',
 '68069',
 '64001',
 '52003',
 '53049',
 '51005',
 '68043',
 '65048',
 '64004',


In [11]:
import requests

In [12]:
from tqdm import tqdm

In [15]:
for id in tqdm(playid):
    url = f"https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={id}"
    r = requests.post(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    aa = soup.find('div', class_='player_basic').text.strip()
    ab = soup.find('h4', id="h4Team").text.strip()
    combined_info = f"{aa}\n소속팀:{ab}"
    player_info_lines = combined_info.split('\n')
    player_dict = {}
    for line in player_info_lines:
        key, value = line.split(':')
        player_dict[key] = value
    if '연봉' in player_dict:
        salary_str = player_dict['연봉'].replace('만원', '').replace(',', '').replace('원', '').replace('달러', '').strip()
    
        # '연봉' 문자열이 비어 있지 않은지 확인합니다.
        if salary_str:
            # '만'이 포함되어 있으면 10000을 곱해 원화로 변환합니다.
            if '만' in player_dict['연봉']:
                salary_in_won = int(salary_str) * 10000
            # '달러'가 포함되어 있으면 환율을 곱해 원화로 변환합니다.
            elif '달러' in player_dict['연봉']:
                exchange_rate = 1292.4
                salary_in_won = int(salary_str) * exchange_rate
            # 숫자만 포함되어 있으면 그대로 사용합니다.
            else:
                salary_in_won = int(salary_str)
    
            # '연봉' 필드를 변환된 값으로 업데이트합니다.
            player_dict['연봉'] = f"{salary_in_won}원"
        else:
            # '연봉' 문자열이 비어 있을 경우 '0'로 설정합니다.
            player_dict['연봉'] = "0"
        player_dict['연봉'] = f"{salary_in_won}"
    df = pd.DataFrame([player_dict])
    df.to_csv(f'./kbo/{id}.csv', index=False, encoding = 'utf-8')



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 921/921 [10:04<00:00,  1.52it/s]
